# Melodic Expectation with Markov Models

In this notebook we will look at Markov Chains for modelling musical expectation.
We have already seen a Markov Model in the class on key estimation with HMMs (Hidden Markov Models).

In [1]:
import os
import numpy as np
import partitura

In [2]:
from rnn import load_data

# To filter out short melodies The minimum number of notes that a sequence should have

min_seq_len = 10
sequences = load_data(min_seq_len)

## Tasks 1; data loading & preparing:
1. check out the content of the variable "sequences", if unclear have a look at the loading function.
2. which musical texture do these sequences exhibit? (https://en.wikipedia.org/wiki/Texture_(music))
3. write a function to derive sequences of pitches from this data.
4. write a function to derive sequences of durations from this data. Modify this to compute inter onset intervals (IOIs; the time between two consecutive onsets). Can you encode rests as well by comparing duration with IOI? 

## Tasks 2; data exploration:

1. compute and draw a histogram of pitches. Modify this to show pitch classes!
2. compute and draw a histogram of IOIs. The input MIDI files are deadpan, i.e. the IOIs in seconds correspond to the notated duration exactly. Look through the IOIs and make an educated guess for some smallest float time unit that could serve as integer smallest time division. Encode the IOIs as multiples of this smallest integer. Which multiples make musical sense?

## Tasks 3; A Markov Chain:

1. choose a data type to model: pitch, pitch class, IOIs, or durations (including or without an encoding for rests). Concatenate all the sequences into one long data sequence.

2. You have now a sequence **X** of symbols from an alphabet **A** (set of possible symbols of your chosen data type):

$$ \mathbf{X} = \{\mathbf{x_0}, \dots, \mathbf{x_n} \mid \mathbf{x}_{i} \in  \mathbf{A} \forall i \in 0, \dots, n \}$$

Compute the empirical conditional probability of seeing any symbol after just having seen any other:

$$ \mathbb{P}(\mathbf{x_i}\mid \mathbf{x_{i-1}}) $$

What is the dimensionality of this probability  given $\lvert A \rvert = d $? Do you recall what this probability was called in the context of HMMs?

3. compute the entropy of the data (only your chosen type). Recall https://en.wikipedia.org/wiki/Entropy_(information_theory)


## Tasks 4; Markov Chain Generation:

1. By computing the probability $ \mathbb{P}(\mathbf{x_i}\mid \mathbf{x_{i-1}}) $ in task 3 you have fully specified a discrete-time finite state space Markov Chain model (https://en.wikipedia.org/wiki/Discrete-time_Markov_chain)! Given an initial symbol "s_0", you can generate the subsequent symbols by sampling from the conditional probability distribution

$$ \mathbb{P}(\mathbf{x_i}\mid \mathbf{x_{i-1}} = \mathbf{s_{0}}) $$

Write a function that samples from a finite state space given an input probability distribution.

2. Use the previously defined function and the Markov Chain to write a sequence generator based on an initial symbol.
3. Start several "walkers", i.e. sampled/generated sequences. Compute the entropy of this generated data and compare it to the entropy in task 3.


## Tasks 5; n-gram Context Model:

1. The Markov Chains used until now have only very limited memory. In fact, they only ever know the last played pitch or duration. Longer memory models can be created by using the conditional probability of any new symbol based on an n-gram context of the symbol (https://en.wikipedia.org/wiki/N-gram):
$$ \mathbb{P}(\mathbf{x_i}\mid \mathbf{x_{i-1}}, \dots, \mathbf{x_{i-n}}) $$

This probability will generally not look like a matrix anymore, but we can easily encode it as a dictionary. Write a function that creates a 3-gram context model from the data sequence **X**!

2. The longer the context, the more data we need to get meaningful or even existing samples for all contexts (note that the number of different contexts grows exponentially with context length). What could we do to approximate the distribution for unseen contexts?

## Tasks 6; multi-type Markov Chains and back to music:

1. To generate a somewhat interesting melody, we want to get a sequence of both pitches and durations. If we encode rests too, we can generate any melody like this. So far our Markov Chains dealt with either pitch or duration/IOI. What could we do to combine them? Describe two approaches and why to choose which one.

2. Implement a simple melody generator with pitch and IOI/duration (simplest; modify taska 4; 2 to a generator of the other type and use them to create independent seuqnces). Write some generated melodies to MIDI files!

## (Tasks 7); more stuff for music:

1. Keys are perceptual centers of gravity in the pitch space, so if we transpose all the input sequences to the same key we can compute empirical pitch distributions within a key!

2. One solution to tasks 5, 2 is to use Prediction by Partial Matching. This is the basis of the most elaborate probabilitstic model ofsymbolic music the Information Dynamics of Music (IDyOM). See references here:
https://researchcommons.waikato.ac.nz/bitstream/handle/10289/9913/uow-cs-wp-1993-12.pdf
https://mtpearce.github.io/idyom/
